# Experiment Notebook: Protocol-Controlled Value Analyses

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: FEI Volatile Liquidity Pool Leverage](#Analysis-1:-FEI-Volatile-Liquidity-Pool-Leverage)
* [Analysis 2: PCV at Risk for Stable Backing Ratio Targets](#Analysis-2:-PCV-at-Risk-for-Stable-Backing-Ratio-Targets)

# Experiment Summary 

The purpose of this notebook is to illustrate and evaluate the effect of a target Stable Backing Ratio and Contractionary Monetary Policy applied to Liquidity Pool protocol-owned liquidity on key system dynamics and KPIs.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code
from experiments.notebooks.helpers.system_metrics import *

In [ ]:
from operator import lt, gt

In [ ]:
# Configure Plotly
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
# simulation_1.model.initial_state.update({})
# simulation_1.model.params.update({})

# Analysis 1: FEI Volatile Liquidity Pool Leverage

This analysis serves to answer the what-if question: What leverage effect does protocol-owned liquidity have on total FEI supply and collateralization of the protocol in different market trends?

In [ ]:
# Analysis-specific setup
simulation_1.engine.drop_substeps = True

simulation_1.model.params.update({
    "liquidity_pool_tvl": np.linspace(start=10e6, stop=200e6, num=3),
    "capital_allocation_fei_deposit_variables": [
        [
            # Toggle on / off to isolate effect of user capital allocation for liquidity provision
            "fei_liquidity_pool_user_deposit",
            "fei_money_market_user_deposit",
            "fei_savings_user_deposit",
            "fei_idle_user_deposit",
        ]
    ]
})

simulation_1.model.params["liquidity_pool_tvl"]

In [ ]:
# Experiment execution
df_1, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

The liquidity pool TVL initial state was swept over three values:

In [ ]:
fig = df_1.plot(y="liquidity_pool_tvl", color="subset")

fig.update_layout(
    title="Liquidity Pool Total Value Locked (TVL) Over Time",
    xaxis_title="Timestamp",
    yaxis_title="TVL (USD)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

A single stochastic Volatile Asset price realisation was used with a negative trend to simulate a bearish market:

In [ ]:
fig = df_1.plot(y="volatile_asset_price", color="subset")

fig.update_layout(
    title="Volatile Asset Price Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    showlegend=False,
)
fig.show()

The change in volatility with increased liquidity is more pronounced when looking at the constant product invariant, and the invariant drives a number of the metrics that follow. The change in volatility of the invariant is specifically caused by the movement of liquidity in and out of the pool by user FEI capital allocation.

In [ ]:
fig = df_1.plot(y="liquidity_pool_invariant", color="subset")

fig.update_layout(
    title="Liquidity Pool Invariant Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Invariant (FEI * Volatile Asset Balance)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

For a specific change in volatile asset price - the larger the invariant, the larger the pool imbalance and resulting minting and redemption required to rebalance the pool:

In [ ]:
fig = df_1.query('timestep < 30').plot(y="fei_minted_redeemed", color="subset")

fig.update_layout(
    title="Liquidity Pool FEI Minted (+ve) / Redeemed (-ve) Over Time",
    xaxis_title="Timestamp",
    yaxis_title="FEI Minted / Redeemed (FEI)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

The volatility in total FEI supply due to arbitrage minting and redemption is more pronounced for deeper liquidity pools:

In [ ]:
df_1["total_fei_supply_norm"] = df_1["total_fei_supply"] / df_1.groupby("subset")["total_fei_supply"].transform('first')

fig = df_1.plot(y="total_fei_supply_norm", color="subset")

fig.update_layout(
    title="Normalised Total FEI Supply Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Normalised Total FEI Supply (Unitless)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
df_1["total_fei_supply_std"] = df_1["total_fei_supply"].rolling(30).std(ddof=0)

fig = df_1.plot(y="total_fei_supply_std", color="subset")

fig.update_layout(
    title="Total FEI Supply Standard Deviation Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Standard Deviation (FEI)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

As expected, the FEI and Volatile Asset liquidity pool balances move in opposite directions and with greater covariance the larger the constant product invariant:

In [ ]:
df_1.groupby("subset")[["fei_liquidity_pool_pcv_deposit_balance", "volatile_liquidity_pool_pcv_deposit_balance"]].cov()

In [ ]:
df_1["fei_liquidity_pool_pcv_deposit_balance_norm"] = (
    df_1["fei_liquidity_pool_pcv_deposit_balance"]
    - df_1.groupby("subset")["fei_liquidity_pool_pcv_deposit_balance"].transform('first')
)
df_1["volatile_liquidity_pool_pcv_deposit_balance_norm"] = (
    df_1["volatile_liquidity_pool_pcv_deposit_balance"]
    - df_1.groupby("subset")["volatile_liquidity_pool_pcv_deposit_balance"].transform('first')
)

fig = df_1.plot(y="fei_liquidity_pool_pcv_deposit_balance_norm", color="subset")

fig.update_layout(
    title="Normalised Liquidity Pool PCV Deposit FEI Balance Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Balance (FEI)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
fig = df_1.plot(y="volatile_liquidity_pool_pcv_deposit_balance_norm", color="subset")

fig.update_layout(
    title="Normalised Liquidity Pool PCV Deposit Volatile Asset Balance Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Balance (Volatile Asset Units)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

Due to higher impermanent loss and leverage of deeper liquidity pools, the collateralization ratio of the protocol is negatively impacted in a market downturn:

In [ ]:
df_1["collateralization_ratio_norm"] = (
    df_1["collateralization_ratio"]
    - df_1.groupby("subset")["collateralization_ratio"].transform('first')
)

fig = df_1.plot(y="collateralization_ratio_norm", color="subset")

fig.update_layout(
    title="Normalised Collateralization Ratio Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio (%)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

# Analysis 2: PCV at Risk for Stable Backing Ratio Targets

The analysis serves to answer the what-if question: What effect does a PCV management strategy targetting a Stable Backing Ratio have on PCV at Risk and collateralization of the protocol? We'll statistically evaluate the efficacy of different policy settings.

### Parameters Sweeped:

We sweep the target stable backing ratio and rebalance direction in the following way:
- Policy 1 (bullish) - keep stable backing ratio <b>below</b> 0.3
- Policy 2 (conservative) - keep stable backing ratio <b>above</b> 0.8

Both policies are executed quarterly over the simulation. The simulation has 100 monte carlo runs.

In [ ]:
parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable backing
    "target_stable_backing_ratio": [0.3, 0.8], # Simulate decrease and increase of stable backing
    "rebalancing_period": [int(365/4)],  # Quarterly rebalancing
}

In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_2.runs = 100

# Override default experiment System Initial State
simulation_2.model.initial_state.update({})

# Override default experiment System Parameters
simulation_2.model.params.update(parameter_overrides)

In [ ]:
# Experiment execution
df_2, exceptions = run(simulation_2)

Volatile asset trajectories for each MC run:

In [ ]:
fig = df_2.query('subset == 0').plot(x='timestamp', y=['volatile_asset_price'], color='run')

fig.update_layout(
    title="Volatile Asset Price Monte Carlo Runs Over Time",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    showlegend=False,
)
fig.show()

## PCV at Risk Computation

Here we compute the empirical distribution of the PCV at Risk (PCVaR) KPI which will inform how likely the PCV portfolio is to lose value over a certain time horizon. For definition see docs.

We set the confidence level (quantile level) at:
$$\alpha=0.95$$

In [ ]:
alpha = 0.95

In [ ]:
df_var = calculate_VaR(df_2, "total_pcv", alpha=alpha, timesteps=1)

In the plots below we see the resulting empirical distribution of the VaR KPI for both policy settings.

In [ ]:
plot_VaR_hist(df_var, 'VaR')

In [ ]:
plot_VaR_hist(df_var, 'q')

As can be seen from the plots, VaR in absolute terms is higher with policy 1 than with policy 2, but the quantile level of PCV returns corresponding to $\alpha=0.95$ is higher in policy 2 than in policy 1.

This is in accordance with intuition - a more conservative policy (higher stable backing) will result in less exposure to volatile asset price movements hence lower potential losses. 

In [ ]:
for subset in df_2['subset'].unique():
    df_var_stats = df_var.query("subset == @subset")[["VaR", "q"]].describe()
    print(f"1-day average PCV at Risk at {100*alpha}th quantile for subset 0: \n {df_var_stats['VaR'].loc['mean']:,.2f} USD")

It is of interest to compute what the likelihood of PCV at risk being greater than a certain level of returns is, to evaluate the resiliency of the policy. Here, we choose a threshold of no more than <b>1%</b> of total PCV at risk per day.

In [ ]:
quantile_return_threshold = -0.01
q_probabilities = calculate_VaR_threshold_probability(df_var, threshold=quantile_return_threshold)

In [ ]:
for subset in q_probabilities.subset.unique():
    print(f"""For Policy {subset + 1}, the 1-Day PCV at Risk is less than {abs(quantile_return_threshold*100):.2f}% with a {100*q_probabilities.query('subset == @subset')['probability'].iloc[0]:.2f}% probability""")

As we can see, since policy 2 is more conservative, it is more effective in having a statistically lower value of PCVaR, implying more contained losses for the protocol.

In [ ]:
df_var_stats_0 = df_var.query("subset == 0")[["VaR", "q"]].describe()
df_var_stats_1 = df_var.query("subset == 1")[["VaR", "q"]].describe()

avg_VaR_delta = df_var_stats_0['VaR'].loc['mean'] - df_var_stats_1['VaR'].loc['mean']
avg_VaR_quantile_delta = df_var_stats_0['q'].loc['mean'] - df_var_stats_1['q'].loc['mean']

In [ ]:
print(f"The Average PCVaR Delta between parameter for policies 1 and 2 is: \n {avg_VaR_delta:,.2f} USD")
print(f"The Average PCVaR Quantile Delta between parameter for policies 1 and 2 is: \n {avg_VaR_quantile_delta:,.4f}")

If you wish to inspect specific realizations of the PCVaR KPI computed on the distributions of PCV returns across policies, the function below can be used with a certain number of runs.

In [ ]:
make_PCVaR_plot(df_2, df_var, 6)

## Effect on Collateralization Ratio

In addition, let us look at the dowstream effect of the target stable backing ratio policy settings on the protocol's collateralization ratio.

In [ ]:
fig = get_averages_by_subset(df_2, ['collateralization_ratio_pct']).plot(
    y='collateralization_ratio_pct',
    color='subset'
)

fig.update_layout(
    title="Collateralization Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio (%)",
    legend=dict(
        title="Subset",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In the plot above we see the average collateralization ratio evolution over 100 monte carlo runs for both policies.

In [ ]:
mu1, mu2 = compute_means(df_2, 'collateralization_ratio')
sr1, sr2 = compute_sr(df_2, 'collateralization_ratio')

In [ ]:
a = mu1 >= mu2
prob = a.sum()/len(a)

b = sr1 >= sr2
prob2 = b.sum()/len(b)

print('The empirical probability of Collateralization Ratio being higher on average with policy 1 than policy 2 is', 100*prob,'%')
print('The empirical probability of Collateralization Ratio Sharpe being higher with policy 1 than policy 2 is', 100*prob2,'%')

Here we compute the probability that, averaged over all monte carlo runs, the mean collateralization ratio is higher in one policy compared to another. We also compute a metric for risk-adjusted return, the sharpe ratio.

As can be seen, with the volatile exposure policy, collateralization ratio is on average higher than with the conservative policy, however its sharpe ratio is <b>never</b> higher (0% probability). This means that when taking risk into consideration, the conservative policy is more effective virtually all the time.

### Conclusion

In this analysis we see how the PCVaR KPI can be leveraged in gauging the statistical soundness of PCV Management for different KPI targets.

Here we expressly chose to compare a very volatile-exposed policy to a highly conservative one, in line with FEI's recent FIPs, to illustrate clear-cut results which under multiple facets all point to the recommendation of the conservative policy.